In [239]:
from funcs import *

In [240]:
df = pd.read_csv("housing/boston_fixed.csv")
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [241]:
X = df.drop("MEDV",axis=1)
y = df["MEDV"]

In [247]:
# NN

class NeuralNetwork:
    def __init__(self, layers, nodes, activations, batchSize=50, activationFn="relu", lr=.1):
        
        self.layers = layers # total number of hidden layers
        
        self.nodes = nodes
        # an int array of size [0, ..., Layers + 1]
        # Nodes[0] shall represent the input size (typically 50)
        # Nodes[Layers + 1] shall represent the output size (typically 1)
        # all other Nodes represent the number of nodes (or width) in the hidden layer i
        
        self.nnodes = [nodes[0], nodes[1], nodes[2]]
        # alternative to nodes where each hidden layer of the nueral network is the same size
        
        self.activations = activations
        # activations[0] and activations[Layers + 1] are left unused
        # activations[i] values are labels indicating the activation function used in layer i
        
        self.batchSize = batchSize
        
        self.activationFn = activationFn
        
        self.lr = lr
      
    # loss over batch
    # x = B * 1 = vector of predictions
    # y = B * 1 = vector of labals
    # output L(x) has size 1
    def jacobian_loss_layer(self, x, y):
        J = {}
        B = len(x)
        for i in range(B):
            J[i] = d_L(x, y, B)
        return pd.series(J)
    
    # x has size B * M
    # output σ(x) has size B * M
    
    def jacobian_activation_layer(self, x, J, func):
        new_x = x.applymap(func).copy()
        J = J * new_x
        return J
    
    def jacobian_dense_layer_input(self, x):
        pass
    
    def jacobian_dense_layer_weights(self, x):
        pass
    
    def forward_pass(self, X_train, y_train):
        
        """ activations : list, length = n_layers - 1
             The ith element of the list holds the values of the ith layer.
        """
        self.activations = []
        self.weights = []
        
        # activations = [None] * 
        df_train = X_train.copy()
        response = y_train.name
        df_train[response] = y_train
        
        df_train_batch = df_train.sample(self.batchSize) # get batch
        X_batch = df_train_batch.drop(response, axis=1)
        X_batch["ones"] = 1 # add ones for bias
        y_batch = df_train_batch[response]
        
        X_batch = X_batch.to_numpy()
        self.y_batch = y_batch.to_numpy()
        
        self.activations.append(X_batch)
        
        M = X_batch.shape[1] # M = number of features
        N = nodes[1] # N = number of nodes in hidden layer
        
        # weights = M * N
        w1 = np.random.normal(size=(M, N)) # initalize weights
        w1 = np.round(w1, 2)
        w1[M-1:] = 0 # initialize biases to 0
        self.weights.append(w1)
        
        h1 = X_batch.dot(w1) # first hidden layer
        h1_activation_function = ACTIVATIONS[self.activationFn]
        h1_activation_function(h1) # h1 is now "activated"
        
        self.activations.append(h1)
        
        w2 = np.random.normal(size=N) # initialize weights
        w2 = np.round(w2, 2)
        w2[N - 1] = 0 # initialize bias to 0
        
        self.z = h1.dot(w2) # z = predictions
        
        w2 = np.reshape(w2, (N, 1))
        self.weights.append(w2)
        
        loss = squared_loss(self.z, y_batch)
        
        self.activations.append(loss)
        
        return self.activations
    
    def backprop(self):
        # loss layer
        J = squared_loss_derivative(self.z, self.y_batch, self.batchSize)
        J = np.reshape(J, (len(J), 1))

        # output layer
        # jacobian w.r.t. weights
        x_t = self.activations[1].T
        J_w2 = x_t.dot(J)
        
        # update jacobian at output layer
        w2_t = self.weights[1].T
        J = np.dot(J, w2_t)
        
        # update jacobian at activation layer
        inplace_relu_derivative(activations[1], J)
        
        # hidden layer
        # jacobian w.r.t. weights
        x_t = activations[0].T
        J_w1 = x_t.dot(J)
        
        # update weights
        self.weights[1] = self.weights[1] - self.lr * J_w2
        self.weights[0] = self.weights[0] - self.lr * J_w1
        
        
        
    def predict(self):
        self.backprop()
        

In [248]:
layers = 1
nodes = [50, 4, 1]
activations = [None, "relu", None]

NN = NeuralNetwork(layers, nodes, activations)
print("layers:", NN.layers)
print("nodes:", NN.nodes)
print("activations:", NN.activations)
print("activationFn:", NN.activationFn)

activations = NN.forward_pass(X, y)
#print("dimensitons:", z.shape)
activations[2]

layers: 1
nodes: [50, 4, 1]
activations: [None, 'relu', None]
activationFn: relu


470094.2142072998

In [249]:
NN.backprop()

10

In [229]:
w2 = NN.weights[1]
w2.shape

(4,)

In [231]:
w2 = np.reshape(w2, (4, 1))
w2.shape

(4, 1)